# Concaténer les output OCR et automatiser une partie de l'encodage
* En entrée : un dossier = toutes les pages .txt d'une océrisation
* En sortie : un fichier texte = concaténation et pré-encodage du texte océrisé
* Script à placer au même niveau que les dossiers d'océrisation

In [1]:
#Import des librairies nécessaires
import os
import re
import glob

In [25]:
def collection_ocr():
#Création d'un dictionnaire id_moreau+liste fichiers texte correspondants
    dic = {}
    for filepath in glob.iglob('*\*.txt'):
        filename = re.split("\\\\", filepath)[-1]
        elems = re.split(".pdf", filename)
        id_moreau = elems[0]
        dic.setdefault(id_moreau, [])
        dic[elems[0]].append(filename)
    return(dic)

def concaten_encod(regex="[^\.].pdf-0?1|2.png_bin.png.txt"):
    dic=collection_ocr()
    p=re.compile(regex, re.IGNORECASE)
    try:
        os.mkdir('sortie_concaten')
    except:
        pass
    for id_moreau, filename in dic.items():
        i=0
        for file in filename:
            m = p.search(file)
            if m:
                pass
            else:
                for filepath in glob.iglob('*\%s' %file):
                    i+=1
                    with open(filepath, "r", encoding='utf-8')as f:
                        lignes=f.readlines()
                    #Encodage des lignes
                    data = [re.sub("^", "<l><lb/>", l) for l in lignes]
                    data = [re.sub("$", "</l>", l) for l in data]
                    #Suppression de l'encodage des lignes vides
                    data = [re.sub("</l>\n</l>", "</l>\n", l) for l in data]
                    data = [re.sub("<l><lb/></l>", "", l) for l in data]
                    texte_encod="".join(data)
                    #Ajout de la balise pb numérotée entre chaque pages
                    with open('sortie_concaten\%s.txt' %id_moreau, 'a', encoding="utf-8")as f:
                        f.write("\n"+'<pb n="%s"/>' %i+"\n"+texte_encod)

# Exemple d'utilisation :

In [26]:
concaten_encod(regex="[^\.].pdf-0?1|2.png_bin.png.txt")

**Les fichiers en sortie se trouvent dans le dossier sortie_concaten**